In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/Data-eq/Train/pneumonia/19a70899-f532-4983-b2d6-3994cc0ef003.png
/kaggle/input/Data-eq/Train/pneumonia/4436b07d-1e7a-4ff1-ab78-4545b025245a.png
/kaggle/input/Data-eq/Train/pneumonia/0e803c36-acc2-466b-b3e7-e22059a0e845.png
/kaggle/input/Data-eq/Train/pneumonia/f990d5db-b516-40f7-8cad-e8d7c2ab5894.png
/kaggle/input/Data-eq/Train/pneumonia/5d1bb85c-cfb6-495a-abeb-479f13420d6a.png
/kaggle/input/Data-eq/Train/pneumonia/e3afc4fc-c1e9-4377-9d6e-cc57ebac1ecd.png
/kaggle/input/Data-eq/Train/pneumonia/41e01e6e-c546-4c6f-866b-46b9d8aaaeb8.png
/kaggle/input/Data-eq/Train/pneumonia/d40e3167-933d-460c-bd22-ef891dc46061.png
/kaggle/input/Data-eq/Train/pneumonia/532f1f82-98b9-4b59-8aeb-1c3a978e15ce.png
/kaggle/input/Data-eq/Train/pneumonia/cd3e57f8-88d4-4112-9422-68e02e66206b.png
/kaggle/input/Data-eq/Train/pneumonia/8bef835d-0aa1-42bf-bd01-955932e850d3.png
/kaggle/input/Data-eq/Train/pneumonia/83722359-bf79-473b-8cdd-f923be16b3c8.png
/kaggle/input/Data-eq/Train/pneumonia/2c120541-fa53-

In [2]:
from PIL import Image
import numpy as np
import os
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

# Define the path to your dataset
dataset_path = "/kaggle/input/Data-eq/"

# Define image size and batch size
image_size = 299
batch_size = 32

# Define the number of classes
num_classes = 4

# Initialize empty lists to store images and labels
X_train = []
y_train = []

# Define the class labels
class_names = ['Normal','Covid','pneumonia','Lung_Opacity']

# Iterate through all classes
for class_name in class_names:
    # Define the path to the current class
    class_path = os.path.join(dataset_path, 'Train', class_name)
    # Iterate through all images in the current class
    for image_name in os.listdir(class_path):
        # Open the image
        img = Image.open(os.path.join(class_path, image_name))
        # Convert the image to a numpy array
        img_array = np.array(img)
        # Append the image to the images list
        X_train.append(img_array)
        # Append the class label to the labels list
        y_train.append(class_names.index(class_name))
        img.close()




In [3]:
# Convert the images and labels lists to numpy arrays
X_train = np.array(X_train)
y_train = np.array(y_train)

# One-hot encode the labels
y_train = to_categorical(y_train, num_classes)

# Reshape the X_train array
X_train = np.reshape (X_train, (X_train.shape[0], 299, 299, 1))

# Split the data into train, validation, and test sets
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2)
X_val, X_test, y_val, y_test = train_test_split(X_val, y_val, test_size=0.5)




In [4]:
from tensorflow.keras.layers import Dropout
from keras import regularizers

In [5]:
# Define the CNN model
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(299, 299, 1), kernel_regularizer=regularizers.l1_l2(l1=0.01, l2=0.01)))
model.add(Conv2D(64,(3,3),activation="relu"))
model.add(MaxPooling2D(pool_size = (2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(64,(3,3),activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(128,(3,3),activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.50))

model.add(Conv2D(128,(3,3),activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.50))

model.add(Flatten())
model.add(Dense(64,activation="relu"))
model.add(Dropout(0.40))
model.add(Dense(num_classes, activation='softmax'))

2023-01-18 13:32:24.087663: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-18 13:32:24.212300: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-18 13:32:24.213920: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-18 13:32:24.216826: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

In [6]:
def add_gaussian_noise(image):
    mean = 0
    var = 0.1
    sigma = var ** 0.5
    gaussian = np.random.normal(mean, sigma, image.shape)
    noisy_image = image + gaussian
    return noisy_image

In [7]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

"""
# Define data augmentation parameters
data_gen_args = dict(featurewise_center=True,
                    featurewise_std_normalization=True,
                     height_shift_range=0.2,
                     zoom_range=0.2,
                     horizontal_flip=True,
                     fill_mode='nearest',
                     preprocessing_function=add_gaussian_noise)

# Create a data generator
data_generator = ImageDataGenerator(**data_gen_args)

# Compute the mean and std of the training data
data_generator.fit(X_train)

# Define the training and validation data generators
train_gen = data_generator.flow(X_train, y_train, batch_size=32)
val_gen = data_generator.flow(X_val, y_val, batch_size=32)

# Train the model
model.fit(train_gen,
                    steps_per_epoch=len(train_gen),
                    validation_data=val_gen,
                    validation_steps=len(val_gen),
                    epochs=3)
"""


model.fit(X_train, y_train, batch_size=batch_size, epochs=5, validation_data=(X_val, y_val))

# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', test_loss)
print('Test accuracy:', test_acc)

2023-01-18 13:32:27.160887: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 1210847144 exceeds 10% of free system memory.
2023-01-18 13:32:28.797854: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 1210847144 exceeds 10% of free system memory.
2023-01-18 13:32:30.003055: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/5


2023-01-18 13:32:32.245495: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


424/424 [==============================] - 61s 120ms/step - loss: 4.3220 - accuracy: 0.5367 - val_loss: 1.4537 - val_accuracy: 0.5989
Epoch 2/5
424/424 [==============================] - 49s 116ms/step - loss: 1.1130 - accuracy: 0.6182 - val_loss: 1.1347 - val_accuracy: 0.6621
Epoch 3/5
424/424 [==============================] - 49s 116ms/step - loss: 1.0232 - accuracy: 0.6407 - val_loss: 0.9469 - val_accuracy: 0.7118
Epoch 4/5
424/424 [==============================] - 49s 115ms/step - loss: 0.9171 - accuracy: 0.6917 - val_loss: 0.8316 - val_accuracy: 0.7484
Epoch 5/5
424/424 [==============================] - 49s 115ms/step - loss: 0.8498 - accuracy: 0.7190 - val_loss: 0.8695 - val_accuracy: 0.7377
Test loss: 0.8775916695594788
Test accuracy: 0.7200236320495605


In [8]:
model.fit(X_train, y_train, batch_size=batch_size, epochs=75, validation_data=(X_val, y_val))

# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', test_loss)
print('Test accuracy:', test_acc)

2023-01-18 13:36:55.470624: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 1210847144 exceeds 10% of free system memory.
2023-01-18 13:36:57.033632: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 1210847144 exceeds 10% of free system memory.


Epoch 1/75
424/424 [==============================] - 49s 116ms/step - loss: 0.8052 - accuracy: 0.7409 - val_loss: 0.7893 - val_accuracy: 0.7732
Epoch 2/75
424/424 [==============================] - 49s 115ms/step - loss: 0.7832 - accuracy: 0.7463 - val_loss: 0.7576 - val_accuracy: 0.7501
Epoch 3/75
424/424 [==============================] - 49s 115ms/step - loss: 0.7329 - accuracy: 0.7609 - val_loss: 0.7127 - val_accuracy: 0.7779
Epoch 4/75
424/424 [==============================] - 49s 115ms/step - loss: 0.7009 - accuracy: 0.7685 - val_loss: 0.6748 - val_accuracy: 0.7726
Epoch 5/75
424/424 [==============================] - 49s 116ms/step - loss: 0.6758 - accuracy: 0.7815 - val_loss: 0.6215 - val_accuracy: 0.7968
Epoch 6/75
424/424 [==============================] - 49s 115ms/step - loss: 0.6650 - accuracy: 0.7798 - val_loss: 0.5997 - val_accuracy: 0.8080
Epoch 7/75
424/424 [==============================] - 49s 115ms/step - loss: 0.6321 - accuracy: 0.7841 - val_loss: 0.6173 - val_ac

In [9]:
model.save('/kaggle/output/model.h5')


In [10]:
model.save_weights('/kaggle/output/weights.h5')


In [11]:
import tensorflow
tensorflow.keras.models.save_model(model, '/kaggle/output/model1', save_format='tf')


2023-01-18 14:38:38.952511: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


In [12]:
model.fit(X_train, y_train, batch_size=batch_size, epochs=2, validation_data=(X_val, y_val))

# Evaluate the model on the test set
#'''test_loss, test_acc = model.evaluate(X_test, y_test, verbose=0)
#print('Test loss:', test_loss)
#print('Test accuracy:', test_acc)'''

2023-01-18 14:38:45.633815: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 1210847144 exceeds 10% of free system memory.


Epoch 1/2
424/424 [==============================] - 49s 116ms/step - loss: 0.2470 - accuracy: 0.9098 - val_loss: 0.4123 - val_accuracy: 0.8659
Epoch 2/2
424/424 [==============================] - 49s 115ms/step - loss: 0.2403 - accuracy: 0.9105 - val_loss: 0.3476 - val_accuracy: 0.8801
